## Выделение ключевых слов

In [1]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

os.chdir('../')

In [2]:
df_keyword = pd.read_csv('data/processed/for_keywords.csv')

### RAKE метод

Применяется к обычному тексту, как так RAKE работает от стоп-слов

In [3]:
from rake_nltk import Rake

rake = Rake(language='russian',
            min_length=2, # минимальное количество слов в фразе
            max_length=10 # максимальное количество слов в фразе
            )

def extract_keywords_rake(text, top_n=5):
    rake.extract_keywords_from_text(text)
    
    # Получаем топ-N фраз
    phrases = rake.get_ranked_phrases()[:top_n]
    
    # Возвращаем как строку
    return ' | '.join(phrases) if phrases else 'Нет ключевых слов'

In [4]:
df_keyword['rake'] = df_keyword['clean_with_stopwords'].apply(lambda x: extract_keywords_rake(x, 5))

df_keyword[['clean_with_stopwords', 'rake']].head(5)

,clean_with_stopwords,rake
0,"обратить взыскание на денежные средства должника ковалев вячеслав геннадьевич г.р., находящиеся на счетах в банках и иных кредитных организациях: юго-западный банк пао сбербанк, бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: пао ""промсвязьбанк"", бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: юго-западный банк пао сбербанк, бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: филиал no банка втб (пао), бик счет: кб ""ренессанс кредит"" (ооо), бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао сбербанк, бик счет: пао росбанк, бик счет: курское отделение n пао сбербанк, бик счет: ао ""альфа-банк"", бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: филиал центральный пао банка ""фк открытие"", бик счет: филиал no банка втб (пао), бик счет: пао ""промсвязьбанк"", бик счет: независимо от остатка денежных средств на счете.","денежные средства должника ковалев вячеслав геннадьевич г | курское отделение n пао сбербанк | фк открытие "", бик счет | филиал no банка втб | филиал no банка втб"
1,"обратить взыскание на денежные средства должника арбузова михаила михайловича г.р., находящиеся на счетах в банках и иных кредитных организациях: ао ""тинькофф банк"", бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: оренбургское отделение n пао сбербанк, бик счет: ао ""тинькофф банк"", бик счет: акб ""абсолют банк"" (пао), бик счет: ;",денежные средства должника арбузова михаила михайловича г | оренбургское отделение n пао сбербанк | оренбургское отделение n пао сбербанк | оренбургское отделение n пао сбербанк | оренбургское отделение n пао сбербанк
2,"судебным приставом-исполнителем обращено взыскание на денежные средства должника кочетова сергея николаевича г.р., находящиеся на счетах в банках и иных кредитных организациях: волгоградское отделение пао сбербанк, бик счет: южный филиал ао ""райффайзенбанк"", бик счет: ао ""тинькофф банк"", бик счет: ао ""тинькофф банк"", бик счет: ао ""тинькофф банк"", бик счет: ао ""райффайзенбанк"", бик счет: пао сбербанк, бик счет: кб ""акрополь"" ао, бик счет: филиал no банка втб (пао), бик счет: южный филиал ао ""райффайзенбанк"", бик счет: южный филиал ао ""райффайзенбанк"", бик счет: волгоградское отделение пао сбербанк, бик счет: волгоградское отделение пао сбербанк, бик счет: волгоградское отделение пао сбербанк, бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: банк гпб (ао), бик счет: филиал no банка втб (пао), бик счет: ;.","денежные средства должника кочетова сергея николаевича г | тинькофф банк "", бик счет | тинькофф банк "", бик счет | тинькофф банк "", бик счет | райффайзенбанк "", бик счет"
3,". вынсено постановление на запрет рег.действий с тс (не указано; не указано; газ г.в.; г/н кан; номер шасси (рамы) номер кузова (прицепа) № двиг: объём двигателя, см куб. .; мощность двигателя, квт .; мощность двигателя, л.с. .; первичный птс (уникальный номер регистрации) ом; наименование технологической операции изменение собственника (владельца); наименование регистрационного документа свидетельство о регистраци

Метод выдаёт самые часто встречающиеся слова, не учитывает важность некоторых слов в начале и конце, хоть и работает со стоп-словами. Даже после лёгкой очистки от шума. То есть не подходит для этого датасета и лучше работает на неформальном неповторяющемся тексте.  
Также можно было вместо удаления частых слов, попробовать найти именованные сущности через **NER** и заменить их на единый токен **[BANK]** или убрать банк, счёт и так далее повторяющееся вовсе. Также можно было попробовать убрать часто встречающиеся фразы в датасете через просмотр через tf-idf, но тут будто дальше тупик.

### YAKE метод

In [5]:
import yake

yake_extractor = yake.KeywordExtractor (
    lan = "ru",     
    n = 10,         # максимальное количество слов в фразе
    dedupLim = 0.8, # порог похожести слов
    top = 3         # количество ключевых слов
    )

df_keyword['yake'] = df_keyword['clean_with_stopwords_with_casing'].apply(
    # lambda x: [kw for kw, score in yake_extractor.extract_keywords(x)]
    lambda x: '; '.join(kw for kw, score in yake_extractor.extract_keywords(x))
)

df_keyword[['clean_with_stopwords_with_casing', 'yake']].head(5)

,clean_with_stopwords_with_casing,yake
0,"Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ г.р., находящиеся на счетах в банках и иных кредитных организациях: ЮГО-ЗАПАДНЫЙ БАНК ПАО СБЕРБАНК, БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ЮГО-ЗАПАДНЫЙ БАНК ПАО СБЕРБАНК, БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: КБ ""Ренессанс Кредит"" (ООО), БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО СБЕРБАНК, БИК счет: ПАО РОСБАНК, БИК счет: КУРСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: АО ""АЛЬФА-БАНК"", БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: ФИЛИАЛ ЦЕНТРАЛЬНЫЙ ПАО БАНКА ""ФК ОТКРЫТИЕ"", БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ПАО ""ПРОМСВЯЗЬБАНК"", БИК счет: независимо от остатка денежных средств на счете.",Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ; взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ; Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ
1,"Обратить взыскание на денежные средства должника Арбузова Михаила Михайловича г.р., находящиеся на счетах в банках и иных кредитных организациях: АО ""ТИНЬКОФФ БАНК"", БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: ОРЕНБУРГСКОЕ ОТДЕЛЕНИЕ N ПАО СБЕРБАНК, БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АКБ ""АБСОЛЮТ БАНК"" (ПАО), БИК счет: ;",Обратить взыскание на денежные средства должника Арбузова Михаила Михайловича; взыскание на денежные средства должника Арбузова Михаила Михайловича; Обратить взыскание на денежные средства должника Арбузова Михаила
2,"Судебным приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея Николаевича г.р., находящиеся на счетах в банках и иных кредитных организациях: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: ЮЖНЫЙ ФИЛИАЛ АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АО ""ТИНЬКОФФ БАНК"", БИК счет: АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: ПАО СБЕРБАНК, БИК счет: КБ ""АКРОПОЛЬ"" АО, БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ЮЖНЫЙ ФИЛИАЛ АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: ЮЖНЫЙ ФИЛИАЛ АО ""РАЙФФАЙЗЕНБАНК"", БИК счет: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: ВОЛГОГРАДСКОЕ ОТДЕЛЕНИЕ ПАО СБЕРБАНК, БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: БАНК ГПБ (АО), БИК счет: ФИЛИАЛ No БАНКА ВТБ (ПАО), БИК счет: ;.",приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея Николаевича; Судебным приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея; обращено взыскание на денежные средства должника Кочетова Сергея Николаевича
3,". вынсено постановление на запрет рег.действий с ТС (Не указано; Не указано; ГАЗ г.в.; г/н КАН; Номер шасси (рамы) Номер кузова (прицепа) № двиг: Объём двигателя, см куб. .; Мощность двигателя, кВт .; Мощность двигателя, л.с. .; Первичный ПТС (уникальный номер

YAKE же сработал намного лучше, показывая начало текста и конец, но также выделяет лишние слова.  

YAKE состоит из 5 разных метрик:  
- Casing (регистр) - В нашем случае важно, связано с началом предложений
- Word Position (позиция, начало и конец предложения) - То же самое, так как основной текст в начале и чуть в конце
- Word Frequency (частота) - Это не очень полезно, так как в нашем случае текст является формальным и кратким
- Word Relatedness to Context (стоп-слова) - Как и RAKE, может помочь, а может нет
- DifSentence (частота появления в предложениях) - Так как текст является формальным и кратким, может быть не очень верным

In [6]:
df_keyword[['clean_with_stopwords_with_casing', 'rake', 'yake']].iloc[5:17]

,clean_with_stopwords_with_casing,rake,yake
5,"В рамках данного ИП установлено, что в отношении недвижимого имущества Земельный участок, адрес: Республика Северная Осетия-Алания, р-н Кировский, ст-ца Змейская, ул. Пашковского, ., кадастровый номер: :, дание, адрес: Республика Северная Осетия-Алания, Кировский район, ст.Змейская, Пашковского ул., д., кадастровый номер: :, вынесено постанвление о запрете действий.","пашковского ул ., д ., кадастровый номер | отношении недвижимого имущества земельный участок | рамках данного ип установлено | ., кадастровый номер | республика северная осетия",отношении недвижимого имущества Земельный участок; недвижимого имущества Земельный участок; Республика Северная Осетия-Алания
6,"Взыскиваемые денежные средства перечислять на следующие реквизиты: Получатель: ООО МКК «ВЕРИТАС» расчетный счет получателя: ИНН: ООО МКК «ВЕРИТАС» – КПП Банк получателя: ООО МКК «ВЕРИТАС» БИК БАНКА: ИНН Банка: КПП Банка: Корсчет: Занесены в исполнительное производство Таким образом, постановление, действие, судебного пристава-исполнителя ОСП по Хунзахскому, Тляратинскому и Шамильскому районам является правомерным.",ооо мкк « веритас » – кпп банк получателя | ооо мкк « веритас » расчетный счет получателя | ооо мкк « веритас » бик банка | шамильскому районам является правомерным | исполнительное производство таким образом,Взыскиваемые денежные средства перечислять на следующие реквизиты; Тляратинскому и Шамильскому районам является правомерным; Занесены в исполнительное производство Таким образом
7,"Судебным приставом-исполнителем ОСП по ЮВАО ГУФССП по г. Москве направлен запрос в ГИБДД по системе поток с целью обнаружения автотранспортного средства CAMRY г.в.; г/н МСР; Номер кузова (прицепа) № двиг: Объём двигателя, см куб. .; Мощность двигателя, кВт .; Мощность двигателя, л.с. .; Первичный ПТС (уникальный номер регистрации) РК; Наименование технологической операции Регистрация ТС (первичная); Наименование регистрационного документа Свидетельство о регистрации транспортного средства (приказ МВД России № -); Серия и номер регистрационного документа Дата выдачи регистрационного документа .; и составления акта описи и ареста имущества.",номер регистрационного документа дата выдачи регистрационного документа | первичная ); наименование регистрационного документа свидетельство | целью обнаружения автотранспортного средства camry г | приказ мвд россии № -); серия | наименование технологической операции регистрация тс,Судебным приставом-исполнителем ОСП по ЮВАО ГУФССП; приставом-исполнителем ОСП по ЮВАО ГУФССП; Судебным приставом-исполнителем ОСП по ЮВАО
8,"Рассмотрев доводы заявителя, Зулькарнеева Алсу Ринатовна признает обстоятельства, изложенные в заявлении (ходатайстве), подлежащими удовлетворению в части: направить постановление об удержании с з/п ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕГИОН-СЕРВИС"" доход В части ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРУДОВЫЕ РЕЗЕРВЫ"" доход ., ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПРОФ РЕЗЕРВ"" доход .г. заявление (ходатайство) Зулькарнеева Алсу Ринатовна признает необоснованным и не подлежащим удовлетворению в связи с нет дохода ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПРОФ РЕЗЕРВ"", ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРУДОВЫЕ РЕЗЕРВЫ"".","зулькарнеева алсу ринатовна признает обстоятельства | зулькарнеева алсу ринатовна признает необоснованным | ходатайстве ), подлежащими удовлетворению | проф резерв "", общество | рассмотрев доводы заявителя",Зулькарнеева Алсу Ринатовна признает необоснованным и не подлежащим удовлетворению; Алсу Ринатовна признает необоснованным и не подлежащим удовлетворению в связи; Зулькарнеева Алсу Ринатовна признает необоснованным и не подлежащим
9,"данные т/с уже внесены в БД АИС мобильного розыска ""Дорожный пристав"", осуществляются еженедельные рейды совместно с органами МРЭО ГИБДД по выявлению залогового т/с, направляются требования в адрес должника для установления местонахождения залогового т/с. Дополнительно осуществляются вых

Хоть и не пробовал метод TextRank, в итоге нету смысла от выделения ключевых слов, так как уже лемматизированные данные подходят под rule-based метод, чтобы определить часть answer, да и выделения ключевых слов не для задачи классификации/генерации